In [1]:
import numpy as np

scores = np.load('results.npy')
print("\nScores:\n", scores.shape)



Scores:
 (4, 10, 10)


In [2]:
mean_scores = np.mean(scores, axis=2).T
print("\nMean scores:\n", mean_scores)


Mean scores:
 [[0.82681159 0.81086957 0.76956522 0.76956522]
 [0.95135149 0.9527852  0.93205036 0.93349435]
 [0.96633748 0.96706204 0.95902748 0.96194719]
 [0.86111111 0.88888889 0.89444444 0.88888889]
 [0.70180375 0.69401579 0.66275783 0.65690094]
 [0.73518519 0.73703704 0.72962963 0.70740741]
 [0.91299799 0.87891348 0.84185111 0.86754527]
 [0.61594203 0.64202899 0.59275362 0.59710145]
 [0.97472563 0.96210483 0.96662572 0.96841114]
 [0.77897793 0.80487805 0.73060395 0.720964  ]]


In [3]:
from sklearn.ensemble import AdaBoostClassifier
from deslib.des.des_knn import DESKNN
from deslib.des.knora_e import KNORAE
from implementedKNORAE import implementedKNORAE

clfs = {
    'AdB' : AdaBoostClassifier(),
    'dkNN': DESKNN(),
    'kne': KNORAE(),
    'ikne' : implementedKNORAE(),
}

In [4]:
from scipy.stats import rankdata
ranks = []
for ms in mean_scores:
    ranks.append(rankdata(ms).tolist())
ranks = np.array(ranks)
print("\nRanks:\n", ranks)


Ranks:
 [[4.  3.  1.  2. ]
 [3.  4.  1.  2. ]
 [3.  4.  1.  2. ]
 [1.  2.5 4.  2.5]
 [4.  3.  2.  1. ]
 [3.  4.  2.  1. ]
 [4.  3.  1.  2. ]
 [3.  4.  1.  2. ]
 [4.  1.  2.  3. ]
 [3.  4.  2.  1. ]]


In [5]:
mean_ranks = np.mean(ranks, axis=0)
print("\nMean ranks:\n", mean_ranks)


Mean ranks:
 [3.2  3.25 1.7  1.85]


In [6]:
from scipy.stats import ranksums

alfa = .05
w_statistic = np.zeros((len(clfs), len(clfs)))
p_value = np.zeros((len(clfs), len(clfs)))

for i in range(len(clfs)):
    for j in range(len(clfs)):
        w_statistic[i, j], p_value[i, j] = ranksums(ranks.T[i], ranks.T[j])

In [7]:
from tabulate import tabulate

headers = list(clfs.keys())
names_column = np.expand_dims(np.array(list(clfs.keys())), axis=1)
w_statistic_table = np.concatenate((names_column, w_statistic), axis=1)
w_statistic_table = tabulate(w_statistic_table, headers, floatfmt=".2f")
p_value_table = np.concatenate((names_column, p_value), axis=1)
p_value_table = tabulate(p_value_table, headers, floatfmt=".2f")
print("\nw-statistic:\n", w_statistic_table, "\n\np-value:\n", p_value_table)


w-statistic:
         AdB    dkNN    kne    ikne
----  -----  ------  -----  ------
AdB    0.00   -0.15   2.68    2.95
dkNN   0.15    0.00   2.72    2.91
kne   -2.68   -2.72   0.00   -0.79
ikne  -2.95   -2.91   0.79    0.00 

p-value:
         AdB    dkNN    kne    ikne
----  -----  ------  -----  ------
AdB    1.00    0.88   0.01    0.00
dkNN   0.88    1.00   0.01    0.00
kne    0.01    0.01   1.00    0.43
ikne   0.00    0.00   0.43    1.00


In [8]:
advantage = np.zeros((len(clfs), len(clfs)))
advantage[w_statistic > 0] = 1
advantage_table = tabulate(np.concatenate(
    (names_column, advantage), axis=1), headers)
print("\nAdvantage:\n", advantage_table)


Advantage:
         AdB    dkNN    kne    ikne
----  -----  ------  -----  ------
AdB       0       0      1       1
dkNN      1       0      1       1
kne       0       0      0       0
ikne      0       0      1       0


In [9]:
significance = np.zeros((len(clfs), len(clfs)))
significance[p_value <= alfa] = 1
significance_table = tabulate(np.concatenate(
    (names_column, significance), axis=1), headers)
print("\nStatistical significance (alpha = 0.05):\n", significance_table)


Statistical significance (alpha = 0.05):
         AdB    dkNN    kne    ikne
----  -----  ------  -----  ------
AdB       0       0      1       1
dkNN      0       0      1       1
kne       1       1      0       0
ikne      1       1      0       0


In [10]:
stat_better = significance * advantage
stat_better_table = tabulate(np.concatenate(
    (names_column, stat_better), axis=1), headers)
print("Statistically significantly better:\n", stat_better_table)

Statistically significantly better:
         AdB    dkNN    kne    ikne
----  -----  ------  -----  ------
AdB       0       0      1       1
dkNN      0       0      1       1
kne       0       0      0       0
ikne      0       0      0       0
